In [2]:
import random
import spacy
from spacy.util import minibatch
from spacy.training.example import Example
from spacy.scorer import Scorer

In [3]:
train_data = [
    ("INV#003281900229170", {"entities": [(0, 19, "RECEIPT NUMBER")]}),
    ("TakeOut Total (incl GST) $26.60", {"entities": [(0, 29, "TOTAL AMOUNT")]}),
    ("Gerbang Alaf Restaurants", {"entities": [(0, 16, "ORG")]}),
    ("DOC#003281900229170", {"entities": [(0, 19, "RECEIPT NUMBER")]}),
    ("ORD#50-REG #19-18/01/2018 10:27:03", {"entities": [(15, 25, "DATE")]}),
    ("Document No TD01167104", {"entities": [(0, 21, "RECEIPT NUMBER")]}),
    ("Total: 9.00", {"entities": [(0, 11, "TOTAL AMOUNT")]}),
    ("Total: $9.00", {"entities": [(0, 11, "TOTAL AMOUNT")]}),
    ("Date: 25/12/2018 8:13:39 PM", {"entities": [(0, 16, "DATE")]}),
    ("Total: ₹9.00", {"entities": [(0, 11, "TOTAL AMOUNT")]}),
    ("19/10/2018 20:49:59 #01", {"entities": [(0, 10, "DATE")]}),
    ("Date of issue 23/10/2019 20:49:59 #01", {"entities": [(0, 24, "DATE")]}),
    ("Invoice Date 19/12/2018 20:49:59 #01", {"entities": [(0, 23, "DATE")]}),
    ("INDAH GIFT & HOME IECO", {"entities": [(0, 21, "ORG")]}),
    ("19-10-2018 20:49:59 #01", {"entities": [(0, 10, "DATE")]}),
    ("TOTAL AMT. RM 60.31", {"entities": [(0, 20, "TOTAL AMOUNT")]}),
    ("2017-03-28 20:49:59 #01", {"entities": [(0, 10, "DATE")]}),
    ("TOTAL AMT. $ 60.31", {"entities": [(0, 18, "TOTAL AMOUNT")]}),
    ("TOTAL RM 33.92", {"entities": [(0, 15, "TOTAL AMOUNT")]}),
    ("Doc No CS00031663 Date 25/12/2018", {"entities": [(0, 17, "RECEIPT NUMBER"), (18, 32, "DATE")]}),
    ("Invoice No CS00031663 Date 25-12-2018", {"entities": [(0, 21, "RECEIPT NUMBER"), (22, 37, "DATE")]}),
    ("Total Sales (Inclusive of GST) 80.90", {"entities": [(0, 35, "TOTAL AMOUNT")]}),
    ("YONGFATT ENTERPRISE", {"entities": [(0, 20, "ORG")]}),
    ("18-11-18 13:58 SH01Z153 T2 R000002902", {"entities": [(0, 8, "DATE")]}),
    ("TOTAL ROUNDED RM 30.90", {"entities": [(0, 21, "TOTAL AMOUNT")]}),
    ("ABC HO TRADING", {"entities": [(0, 14, "ORG")]}),
    ("Total Amount: 31.00", {"entities": [(0, 19, "TOTAL AMOUNT")]}),
    ("Date 09/01/2019 8:01:11 PM", {"entities": [(0, 15, "DATE")]}),
    ("Total Amount: $31.00", {"entities": [(0, 20, "TOTAL AMOUNT")]}),
    ("SOON HUAT MACHINERY ENTERPRISE", {"entities": [(0, 30, "ORG")]}),
    ("Doc No. CS00004040 Date 11/01/2019", {"entities": [(0, 18, "RECEIPT NUMBER"), (19, 33, "DATE")]}),
    ("Total Sales INR 327.00", {"entities": [(0, 22, "TOTAL AMOUNT")]}),
    ("Total Sales $ 327.00", {"entities": [(0, 20, "TOTAL AMOUNT")]}),
    ("Total Sales USD 327.00", {"entities": [(0, 22, "TOTAL AMOUNT")]}),
    ("Total Sales 327.00", {"entities": [(0, 18, "TOTAL AMOUNT")]}),
    ("Receipt#: Cs00082552", {"entities": [(0, 20, "RECEIPT NUMBER")]}),
    ("Total (RM): 112.46", {"entities": [(0, 18, "TOTAL AMOUNT")]}),
    ("Total (INR): 112.46", {"entities": [(0, 19, "TOTAL AMOUNT")]}),
    ("Total (USD): 112.46", {"entities": [(0, 19, "TOTAL AMOUNT")]}),
    ("Salesperson: Date: 12/02/2018", {"entities": [(13, 29, "DATE")]}),
    ("SAM SAM TRADING CO", {"entities": [(0, 18, "ORG")]}),
    ("TOTAL RM 14.10", {"entities": [(0, 14, "TOTAL AMOUNT")]}),
    ("Friday 29-12-2017 Time: 20:17", {"entities": [(7, 17, "DATE")]}),
    ("Good Trust Trading LLC", {"entities": [(0, 22, "ORG")]}),
    ("Friday 2019/06/08 Time: 20:17", {"entities": [(7, 17, "DATE")]}),
    ("Final Amount (incl. Tax) ₹123.45", {"entities": [(0, 32, "TOTAL AMOUNT")]}),
    ("Final Amount (incl. Tax) INR 123.45", {"entities": [(0, 35, "TOTAL AMOUNT")]}),
    ("Grand Total $2,538.968", {"entities": [(0, 22, "TOTAL AMOUNT")]}),
    ("In Words: Omani Rial Two Thousand Five Hundred Thirty Eight and 968/1000 Only", {"entities": [(9, 70, "TOTAL AMOUNT")]}),
    ("Final Amount (incl. Tax) $123.45", {"entities": [(0, 35, "TOTAL AMOUNT")]}),
    ("Fo Sood Trust Trading LLC", {"entities": [(0, 29, "ORG")]}),
    ("RESTORAN HASSAN BISTRO", {"entities": [(0, 22, "ORG")]}),
    ("Invoice No: 987654321012345", {"entities": [(0, 27, "RECEIPT NUMBER")]}),
    ("Receipt No: 987654321012345", {"entities": [(0, 27, "RECEIPT NUMBER")]}),
    ("Final Amount (incl. Tax) 123.45", {"entities": [(0, 31, "TOTAL AMOUNT")]}),
    ("Final Amount (incl. Tax) $123.45", {"entities": [(0, 32, "TOTAL AMOUNT")]}),
    ("MegaMart Superstore", {"entities": [(0, 20, "ORG")]}),
    ("ORD#21-REG #05-10/03/2022 14:50:10", {"entities": [(15, 25, "DATE")]}),
    ("Final Amount (incl. Tax) USD 123.45", {"entities": [(0, 35, "TOTAL AMOUNT")]}),
    ("TOTAL PAYABLE: $745.00", {"entities": [(0, 22, "TOTAL AMOUNT")]}),
    ("Document No: TX981723", {"entities": [(0, 21, "RECEIPT NUMBER")]}),
    ("Total: 150.75", {"entities": [(0, 13, "TOTAL AMOUNT")]}),
    ("Date: 08/11/2021 19:30:45", {"entities": [(0, 16, "DATE")]}),
    ("E-Mart Wholesale", {"entities": [(0, 17, "ORG")]}),
    ("15/09/2020 12:15:30 #05", {"entities": [(0, 10, "DATE")]}),
    ("TOTAL PAYABLE: 745.00", {"entities": [(0, 22, "TOTAL AMOUNT")]}),
    ("TOTAL DUE: 49.99", {"entities": [(0, 16, "TOTAL AMOUNT")]}),
    ("TOTAL DUE: INR 49.99", {"entities": [(0, 20, "TOTAL AMOUNT")]}),
    ("TOTAL DUE: USD 49.99", {"entities": [(0, 20, "TOTAL AMOUNT")]}),
    ("Doc No: AB123456 Date: 12/06/2019", {"entities": [(0, 16, "RECEIPT NUMBER"), (17, 33, "DATE")]}),
    ("Total (inclusive of Tax): $210.99", {"entities": [(0, 33, "TOTAL AMOUNT")]}),
    ("Total (inclusive of Tax): 210.99", {"entities": [(0, 32, "TOTAL AMOUNT")]}),
    ("Global Traders Ltd.", {"entities": [(0, 19, "ORG")]}),
    ("22-08-2021 10:30 AM", {"entities": [(0, 10, "DATE")]}),
    ("TOTAL BALANCE: USD 1,150.40", {"entities": [(0, 27, "TOTAL AMOUNT")]}),
    ("TOTAL BALANCE: 1,150.40", {"entities": [(0, 23, "TOTAL AMOUNT")]}),
    ("XYZ Electronics", {"entities": [(0, 15, "ORG")]}),
    ("Grand Total  USD 980.50", {"entities": [(0, 23, "TOTAL AMOUNT")]}),
    ("Grand Total: INR 980.50", {"entities": [(0, 23, "TOTAL AMOUNT")]}),
    ("Grand Total: ₹ 980.50", {"entities": [(0, 21, "TOTAL AMOUNT")]}),
    ("Grand Total: 980.50", {"entities": [(0, 19, "TOTAL AMOUNT")]}),
    ("Invoice Date 05/04/2022 16:45:12", {"entities": [(0, 23, "DATE")]}),
    ("Date 05/04/2022 16:45:12", {"entities": [(0, 15, "DATE")]}),
    ("Super Market Express", {"entities": [(0, 20, "ORG")]}),
    ("Doc No. XY98765432 Date: 14/07/2020", {"entities": [(0, 18, "RECEIPT NUMBER"), (19, 35, "DATE")]}),
    ("Total Purchase: ₹450.60", {"entities": [(0, 23, "TOTAL AMOUNT")]}),
    ("Total Purchase: 450.60", {"entities": [(0, 22, "TOTAL AMOUNT")]}),
    ("Receipt#: ZX321654987", {"entities": [(0, 21, "RECEIPT NUMBER")]}),
    ("Total (Amount): 189.75", {"entities": [(0, 22, "TOTAL AMOUNT")]}),
    ("Total (Amount): $189.75", {"entities": [(0, 23, "TOTAL AMOUNT")]}),
    ("Total (Amount): INR 189.75", {"entities": [(0, 26, "TOTAL AMOUNT")]}),
    ("Total (Amount): USD 189.75", {"entities": [(0, 26, "TOTAL AMOUNT")]}),
    ("Transaction Date: 03/12/2021", {"entities": [(0, 28, "DATE")]}),
    ("Invoice Date: 03/12/2021", {"entities": [(0, 24, "DATE")]}),
    ("ABC & Co. Stores", {"entities": [(0, 16, "ORG")]}),
    ("TOTAL PAID: 22.90", {"entities": [(0, 17, "TOTAL AMOUNT")]}),
    ("TOTAL PAID: $22.90", {"entities": [(0, 18, "TOTAL AMOUNT")]}),
    ("Thursday 17-06-2021 Time: 18:45", {"entities": [(9, 19, "DATE")]}),
    ("Premium Shopping Mall", {"entities": [(0, 21, "ORG")]}),
    ("Total Bill: $3,999.99", {"entities": [(0, 21, "TOTAL AMOUNT")]}),
    ("Total Bill: INR 3,999.99", {"entities": [(0, 24, "TOTAL AMOUNT")]}),
    ("e17-06-2018 14:0729", {"entities": [(1, 11, "DATE")]}),
    ("Total Bill: ₹3,999.99", {"entities": [(0, 21, "TOTAL AMOUNT")]}),
    ("Total Bill: USD 3,999.99", {"entities": [(0, 24, "TOTAL AMOUNT")]}),
    ("Total Bill (USD): 3,999.99", {"entities": [(0, 26, "TOTAL AMOUNT")]}),
    ("e2020-10-19 14:0729", {"entities": [(1, 11, "DATE")]}),
    ("Amount in Words: Three Thousand Nine Hundred Ninety-Nine and 99/100 Only", {"entities": [(16, 78, "TOTAL AMOUNT")]}),
    ("Bright Retailers Ltd.", {"entities": [(0, 20, "ORG")]}),
    ("RESTAURANT GOURMET DELIGHT", {"entities": [(0, 26, "ORG")]}),
    ("DATE OF PURCHASE: 30/12/2020", {"entities": [(0, 28, "DATE")]}),
    ("DATE OF ISSUE: 30/12/2020", {"entities": [(0, 25, "DATE")]}),
    ("Cash InvNo.:7006F710", {"entities": [(0, 20, "RECEIPT NUMBER")]}),
    ("DD:08/10/2017 12:11", {"entities": [(0, 13, "DATE")]}),
    ("DD:2017/08/13 12:11", {"entities": [(0, 13, "DATE")]}),
]

In [4]:
eval_data = [
    ("27/03/2018 NoCS-20243", {"entities": [(0, 10, "DATE")]}),
    ("Total Amt Payable: 12.00", {"entities": [(0, 24, "TOTAL AMOUNT")]}),
    ("27-03-2018 NoCS-20243", {"entities": [(0, 10, "DATE")]}),
    ("2019-05-01 NoCS-20243", {"entities": [(0, 10, "DATE")]}),
    ("invoice No.: CS 10012", {"entities": [(0, 21, "RECEIPT NUMBER")]}),
    ("Date: 20/03/201813:01", {"entities": [(0, 16, "DATE")]}),
    ("doc No.: CS 10012", {"entities": [(0, 21, "RECEIPT NUMBER")]}),
    ("Date: 20/03/201813:01", {"entities": [(0, 16, "DATE")]}),
    ("Total incl.of GST 102.00", {"entities": [(0, 24, "TOTAL AMOUNT")]}),
    ("Date: 20-03-201813:01", {"entities": [(0, 16, "DATE")]}),
    ("Total Ant Rounded 102.00", {"entities": [(0, 24, "TOTAL AMOUNT")]}),
    ("GL HANDICRAFT &TAILORING", {"entities": [(0, 24, "ORG")]}),
    ("ONE ONE THREE SEAFOOD RESTAURANT SDN BHD", {"entities": [(0, 40, "ORG")]}),
    ("Date 23-06-201820:52:16", {"entities": [(0, 15, "DATE")]}),
    ("TotalInclusive of GST 179.50", {"entities": [(0, 28, "TOTAL AMOUNT")]}),
    ("TOTAL: $179.50", {"entities": [(0, 14, "TOTAL AMOUNT")]}),
    ("TOTAL: ₹179.50", {"entities": [(0, 14, "TOTAL AMOUNT")]}),
    ("Bi11#V001-541021 Order#141126", {"entities": [(0, 16, "RECEIPT NUMBER")]}),
    ("Bi11#V001-540835 Order#140933", {"entities": [(0, 16, "RECEIPT NUMBER")]}),
    ("Date20-06-201821:36:11", {"entities": [(0, 14, "DATE")]}),
    ("e17/06/2018 14:0729", {"entities": [(1, 11, "DATE")]}),
    ("Date20-06-201821:36:11", {"entities": [(0, 14, "DATE")]}),
    ("TotalInclusive ofGST: ₹38.00", {"entities": [(0, 28, "TOTAL AMOUNT")]}),
    ("TOTAL: $38.00", {"entities": [(0, 13, "TOTAL AMOUNT")]}),
    ("INV# 1074908 Cashier:Thandan", {"entities": [(0, 12, "RECEIPT NUMBER")]}),
    ("TotalInclusive of GsT 58.00", {"entities": [(0, 27, "TOTAL AMOUNT")]}),
    ("TOTAL: 58.00", {"entities": [(0, 27, "TOTAL AMOUNT")]}),
    ("RESTORAN WAN SHENG", {"entities": [(0, 18, "ORG")]}),
    ("INV NO.1074908 Cashier:Thandan", {"entities": [(0, 14, "RECEIPT NUMBER")]}),
    ("INV# 1074908 Cashier:Thandan", {"entities": [(0, 12, "RECEIPT NUMBER")]}),
    ("Date 02-04-201815:14:07", {"entities": [(0, 15, "DATE")]}),
    ("Date 02-04-201815:14:07", {"entities": [(0, 15, "DATE")]}),
    ("TOTAL: 6.70", {"entities": [(0, 15, "TOTAL AMOUNT")]}),
    ("MAKASSAR FRESH MARKET SDN BHD", {"entities": [(0, 29, "ORG")]}),
    ("Total Payable: 46.06", {"entities": [(0, 20, "TOTAL AMOUNT")]}),
    ("DD:08-10-2017 12:11", {"entities": [(0, 20, "TOTAL AMOUNT")]}),
    ("DD:2017-07-02 12:11", {"entities": [(0, 20, "TOTAL AMOUNT")]}),
]

In [5]:
# nlp = spacy.load('en_core_web_md')
nlp = spacy.blank("en")

In [6]:
if 'ner' not in nlp.pipe_names:
    ner = nlp.add_pipe('ner')
else:
    ner = nlp.get_pipe('ner')


In [7]:
nlp.pipe_names

['ner']

In [8]:
for _, annotations in train_data:
    for ent in annotations['entities']:
        if ent[2] not in ner.labels:
            ner.add_label(ent[2])
            

In [10]:
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']


with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.initialize()
    
    epochs = 50
    for epoch in range(epochs):
        random.shuffle(train_data)
        losses = {}
        batches = minibatch(train_data, size=8)
        for batch in batches:
            examples = []
            for text, annotations in batch:
                doc = nlp.make_doc(text)
                example = Example.from_dict(doc, annotations)
                examples.append(example)
                
            nlp.update(examples, drop=0.2, losses=losses)
        print(f"Epoch {epoch + 1} | Losses : {losses}")
        
nlp.to_disk('invoice_ner_model')

trained_model = spacy.load('invoice_ner_model')

# eval_data = train_data

scorer = Scorer()
examples = []
for text, annotations in eval_data:
    doc = trained_model.make_doc(text)
    example = Example.from_dict(doc, annotations)
    example.predicted = trained_model(text)
    examples.append(example)
    
scores = scorer.score(examples)

print("\nModel Evaluation:")
print(f"Precision: {scores['ents_p']:.3f}")
print(f"Recall: {scores['ents_r']:.3f}")
print(f"F1-score: {scores['ents_f']:.3f}")

testing_data = [
    "Receipt #: 9876543210",
    "Total: $45.67",
    "TOTAL AMOUNT USD 150.89",
    "Invoice No. ZX-2024123",
    "Document No: 002345678",
    "Bill Date: 15-07-2022",
    "Invoice Date: 2023/11/05",
    "Receipt No.: CS2300198 Date : 23/01/2025",
    "Total Payable : $240",
    "Subtotal: ₹180",
    "Good Trust Trading LLC",
    "Dat 31.05.2023",
    "Grand Total 2,538.968",
    "In Words:Omani Ria Two Thousand Five Hundred Thirty Eight and 968/1000 Only",
    "Total 5250.000",
    "Total R.O-Five Thousand Two Hundred Fifty Only",
    "TOTAL 1,380.000",
    "Al noor Projects Engineering & Trading LLC"
]

for text in testing_data:
    doc = trained_model(text)
    print(f"Text : {text}")
    print(f"Entities : {[(ent.text, ent.label_) for ent in doc.ents]}")
    print()

d:\InvoiceFieldExtraction\source\.venv\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Gerbang Alaf Restaurants" with entities "[(0, 16, 'ORG')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
d:\InvoiceFieldExtraction\source\.venv\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Doc No. CS00004040 Date 11/01/2019" with entities "[(0, 18, 'RECEIPT NUMBER'), (19, 33, 'DATE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
d:\InvoiceFieldExtraction\source\.venv\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "MegaMart Superstore" 

Epoch 1 | Losses : {'ner': 334.7369252536446}
Epoch 2 | Losses : {'ner': 264.47940373072646}
Epoch 3 | Losses : {'ner': 225.64588129715028}
Epoch 4 | Losses : {'ner': 134.78388556375648}
Epoch 5 | Losses : {'ner': 113.49194270937971}
Epoch 6 | Losses : {'ner': 87.6654501446605}
Epoch 7 | Losses : {'ner': 113.24050861510293}
Epoch 8 | Losses : {'ner': 90.86408317570637}
Epoch 9 | Losses : {'ner': 59.1118423257984}
Epoch 10 | Losses : {'ner': 39.89605561312665}
Epoch 11 | Losses : {'ner': 20.225538712824758}
Epoch 12 | Losses : {'ner': 9.622437736610731}
Epoch 13 | Losses : {'ner': 6.102654810859293}
Epoch 14 | Losses : {'ner': 0.9647915884175859}
Epoch 15 | Losses : {'ner': 0.8585324503295717}
Epoch 16 | Losses : {'ner': 0.08477502438438715}
Epoch 17 | Losses : {'ner': 0.040227086072440625}
Epoch 18 | Losses : {'ner': 0.05185362797092661}
Epoch 19 | Losses : {'ner': 2.832144290782946}
Epoch 20 | Losses : {'ner': 3.806192473919613}
Epoch 21 | Losses : {'ner': 2.14880416800443}
Epoch 22 |

d:\InvoiceFieldExtraction\source\.venv\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Date: 20/03/201813:01" with entities "[(0, 16, 'DATE')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
d:\InvoiceFieldExtraction\source\.venv\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "doc No.: CS 10012" with entities "[(0, 21, 'RECEIPT NUMBER')]". Use `spacy.training.offsets_to_biluo_tags(nlp.make_doc(text), entities)` to check the alignment. Misaligned entities ('-') will be ignored during training.
  warnings.warn(
d:\InvoiceFieldExtraction\source\.venv\Lib\site-packages\spacy\training\iob_utils.py:149: UserWarning: [W030] Some entities could not be aligned in the text "Date: 20-03-201813:01" with entities "[(0, 16, 'DATE')]". 


Model Evaluation:
Precision: 0.682
Recall: 0.652
F1-score: 0.667
Text : Receipt #: 9876543210
Entities : [('Receipt #: 9876543210', 'RECEIPT NUMBER')]

Text : Total: $45.67
Entities : [('Total: $45.67', 'TOTAL AMOUNT')]

Text : TOTAL AMOUNT USD 150.89
Entities : [('TOTAL AMOUNT USD 150.89', 'TOTAL AMOUNT')]

Text : Invoice No. ZX-2024123
Entities : [('Invoice No. ZX-2024123', 'RECEIPT NUMBER')]

Text : Document No: 002345678
Entities : [('Document No: 002345678', 'RECEIPT NUMBER')]

Text : Bill Date: 15-07-2022
Entities : [('Date: 15-07-2022', 'DATE')]

Text : Invoice Date: 2023/11/05
Entities : [('Invoice Date: 2023/11/05', 'DATE')]

Text : Receipt No.: CS2300198 Date : 23/01/2025
Entities : [('Receipt No.: CS2300198', 'RECEIPT NUMBER'), ('Date : 23/01/2025', 'DATE')]

Text : Total Payable : $240
Entities : [('Total Payable : $240', 'TOTAL AMOUNT')]

Text : Subtotal: ₹180
Entities : [('Subtotal: ₹180', 'TOTAL AMOUNT')]

Text : Good Trust Trading LLC
Entities : [('Good Trust Trading L